In [1]:
import equinox as eqx
import gym
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import optax
import sys
import timeit
import wandb

from argparse import Namespace
from functools import partial
from jax import grad, jit, vmap
from typing import Sequence

from jax_learning.agents.rl_agents import EpsilonGreedyAgent
from jax_learning.buffers.ram_buffers import NextStateNumPyBuffer
from jax_learning.buffers.utils import batch_flatten, to_jnp
from jax_learning.common import init_wandb
from jax_learning.constants import DISCRETE
from jax_learning.envs.wrappers.image import HWC2CHW
from jax_learning.learners.q_learning import QLearning
from jax_learning.models.q_functions import SoftmaxQ, NatureQ
from jax_learning.rl_utils import interact, evaluate

In [2]:
init_wandb(project="test_jax_rl", group="breakout-dqn_test", mode="disabled")

In [3]:
cfg_dict = {
    # Environment setup
    "env": "Breakout-v4",
    "seed": 0,
    "render": False,
    # Experiment progress
    "load_step": 0,
    "log_interval": 5000,
    "checkpoint_frequency": 0,
    "save_path": None,
    # Learning hyperparameters
    "max_timesteps": 1000000,
    "buffer_size": 1000000,
    "buffer_warmup": 1000,
    "num_gradient_steps": 1,
    "batch_size": 64,
    "lr": 3e-4,
    "max_grad_norm": 10.0,
    "gamma": 0.99,
    "update_frequency": 4,
    "target_update_frequency": 1,
    "tau": 0.005,  # This is for polyak averaging of target network
    "omega": 1.0,  # This is for residual gradient: 1 for semi-gradient
    # Epsilon greedy hyperparameters
    "init_eps": 1.0,
    "min_eps": 0.02,
    "eps_decay": 0.9999,
    "eps_warmup": 1000,
    # Normalization
    "normalize_obs": False,
    "normalize_value": False,
    # Model architecture
    "hidden_dim": 64,
    "num_hidden": 1,
    # Evaluation
    "evaluation_frequency": 5000,
    "eval_cfg": {
        "num_episodes": 10,
        "seed": 1,
        "render": True,
    },
}
cfg = Namespace(**cfg_dict)
eval_cfg = Namespace(**cfg.eval_cfg)
wandb.config = cfg_dict

In [4]:
np.random.seed(cfg.seed)

In [5]:
env = gym.make(cfg.env)
env = HWC2CHW(env, scale=255)

A.L.E: Arcade Learning Environment (version 0.8.0+919230b)
[Powered by Stella]
/Users/chanb/research/personal/jax_learning/.venv/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
cfg.obs_dim = env.observation_space.shape
cfg.act_dim = (env.action_space.n,)
cfg.action_space = DISCRETE

In [7]:
cfg.h_state_dim = (1,)
cfg.rew_dim = (1,)

In [8]:
cfg.buffer_rng = np.random.RandomState(cfg.seed)
cfg.env_rng = np.random.RandomState(cfg.seed)
cfg.agent_key, cfg.model_key = jrandom.split(jrandom.PRNGKey(cfg.seed), num=2)
eval_cfg.env_rng = np.random.RandomState(eval_cfg.seed)
cfg.evaluation_cfg = eval_cfg

In [9]:
cfg

Namespace(env='Breakout-v4', seed=0, render=False, load_step=0, log_interval=5000, checkpoint_frequency=0, save_path=None, max_timesteps=1000000, buffer_size=1000000, buffer_warmup=1000, num_gradient_steps=1, batch_size=64, lr=0.0003, max_grad_norm=10.0, gamma=0.99, update_frequency=4, target_update_frequency=1, tau=0.005, omega=1.0, init_eps=1.0, min_eps=0.02, eps_decay=0.9999, eps_warmup=1000, normalize_obs=False, normalize_value=False, hidden_dim=64, num_hidden=1, evaluation_frequency=5000, eval_cfg={'num_episodes': 10, 'seed': 1, 'render': True}, obs_dim=(3, 210, 160), act_dim=(4,), action_space='discrete', h_state_dim=(1,), rew_dim=(1,), buffer_rng=RandomState(MT19937) at 0x16A2F2C40, env_rng=RandomState(MT19937) at 0x16A2F2D40, agent_key=DeviceArray([4146024105,  967050713], dtype=uint32), model_key=DeviceArray([2718843009, 1272950319], dtype=uint32), evaluation_cfg=Namespace(num_episodes=10, seed=1, render=True, env_rng=RandomState(MT19937) at 0x110F48940))

In [10]:
Q = "q"

buffer = NextStateNumPyBuffer(
    buffer_size=cfg.buffer_size,
    obs_dim=cfg.obs_dim,
    h_state_dim=cfg.h_state_dim,
    act_dim=(1,) if cfg.action_space == DISCRETE else cfg.act_dim,
    rew_dim=cfg.rew_dim,
    rng=cfg.buffer_rng,
)

model = {
    Q: SoftmaxQ(
        q_function=NatureQ(
            in_channel=cfg.obs_dim[0],
            height=cfg.obs_dim[1],
            width=cfg.obs_dim[2],
            out_dim=cfg.act_dim,
            hidden_dim=cfg.hidden_dim,
            num_hidden=cfg.num_hidden,
            key=cfg.model_key,
        )
    ),
}

target_model = {
    Q: SoftmaxQ(
        q_function=NatureQ(
            in_channel=cfg.obs_dim[0],
            height=cfg.obs_dim[1],
            width=cfg.obs_dim[2],
            out_dim=cfg.act_dim,
            hidden_dim=cfg.hidden_dim,
            num_hidden=cfg.num_hidden,
            key=cfg.model_key,
        )
    ),
}

opt_transforms = [optax.scale_by_rms(), optax.scale(-cfg.lr)]
if cfg.max_grad_norm:
    opt_transforms.insert(0, optax.clip_by_global_norm(cfg.max_grad_norm))
opt = {Q: optax.chain(*opt_transforms)}

learner = QLearning(
    model=model, target_model=target_model, opt=opt, buffer=buffer, cfg=cfg
)

agent = EpsilonGreedyAgent(
    model=model,
    model_key=Q,
    buffer=buffer,
    learner=learner,
    init_eps=cfg.init_eps,
    min_eps=cfg.min_eps,
    eps_decay=cfg.eps_decay,
    eps_warmup=cfg.eps_warmup,
    action_space=DISCRETE,
    action_dim=cfg.act_dim[0],
    key=cfg.agent_key,
)

In [11]:
%wandb

In [12]:
interact(env, agent, cfg)

  0%|                         | 972/1000000 [00:02<43:18, 384.40it/s]

sample: 0.0065018750028684735
put to device: 0.0005341670184861869
update Q: 1.3831927920109592
sample: 0.005517332989256829
put to device: 0.00023899998632259667
update Q: 0.37944674998288974
sample: 0.00501412499579601
put to device: 0.0002509589830879122
update Q: 0.38247829201282
sample: 0.005646042001899332
put to device: 0.00023437500931322575


  0%|                       | 1013/1000000 [00:05<5:43:36, 48.46it/s]

update Q: 0.3772512919967994
sample: 0.0059889999974984676
put to device: 0.0002976669929921627
update Q: 0.3797273749951273
sample: 0.0050945420225616544
put to device: 0.0002346250112168491
update Q: 0.38622470799600706
sample: 0.005729041004087776
put to device: 0.00023004200193099678
update Q: 0.37737887498224154
sample: 0.005105666990857571
put to device: 0.0002213749976363033
update Q: 0.37763087498024106
sample: 0.004754624998895451
put to device: 0.00022183300461620092
update Q: 0.37970033401506953
sample: 0.005145290982909501
put to device: 0.00023616699036210775
update Q: 0.39032970799598843
sample: 0.005505083972821012
put to device: 0.0002307090035174042


  0%|                      | 1042/1000000 [00:08<10:19:28, 26.88it/s]

...

update Q: 0.4055310410039965
sample: 0.004680166981415823
put to device: 0.0002831250021699816


  0%|                      | 1225/1000000 [00:26<28:47:28,  9.64it/s]

update Q: 0.39320695798960514
sample: 0.004726749990368262
put to device: 0.00022833398543298244


  0%|                       | 1228/1000000 [00:27<6:11:06, 44.86it/s]


KeyboardInterrupt: 

In [ ]:
wandb.finish()